In [ ]:
# !pip install segmentation-models-pytorch
# !pip install pytorch_msssim
# !pip install scikit-image
# !pip install torch
# !pip install -r requirements.txt
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

  Using cached segmentation_models_pytorch-0.3.4-py3-none-any.whl.metadata (30 kB)
  Using cached efficientnet_pytorch-0.7.1-py3-none-any.whl
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached pretrainedmodels-0.7.4-py3-none-any.whl
  Using cached timm-0.9.7-py3-none-any.whl.metadata (58 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached munch-4.0.0-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached safetensors-0.4.5-cp39-none-win_amd64.whl.metadata (3.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp39-cp39-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached segmentation_models_pytorch-0.3.4-py3-none-any.w

In [31]:
import random
import numpy as np
import os

import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import cv2

import zipfile


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [32]:
CFG = {
    'EPOCHS':2,
    'LEARNING_RATE':3e-4,
    # 'BATCH_SIZE':16,
    'BATCH_SIZE':16,
    'SEED':42
}

In [33]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [34]:
#저장된 이미지 쌍을 동시에 로드 

class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.damage_files = sorted(os.listdir(damage_dir))
        self.origin_files = sorted(os.listdir(origin_dir))

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img}

In [35]:
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms

# 경로 설정
origin_dir = 'data/train_gt'  # 원본 이미지 폴더 경로
damage_dir = 'data/train_input'  # 손상된 이미지 폴더 경로
test_dir = 'data/test_input'     # test 이미지 폴더 경로

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 전체 데이터셋 생성
dataset = CustomDataset(damage_dir=damage_dir, origin_dir=origin_dir, transform=transform)

# 데이터셋을 학습과 검증으로 나누기 (예: 80% 학습, 20% 검증)
validation_ratio = 0.2
train_size = int((1 - validation_ratio) * len(dataset))
val_size = len(dataset) - train_size
training_dataset, validation_dataset = random_split(dataset, [train_size, val_size])


# 1. gpt가 짜준 ssimscore랑 공모전 metric 코드랑 뭐가 다른지 비교

# 2. bceloss랑 poerceptual loss 들어간 새로운 combined loss로 테스트해보기

# # 편법
# 3. 1)
# train_sizeL 29603 
# test: 100
# 우리가 갖ㄱ도 있는 최고의의 모데리: 0.5
# test=> 대충정답: 컬러사진

# 2). 그다음학습 때
# training_dataset =>   (0.8*dataset) + + test=> 대충정답: 컬러사진
# validation_dataset: (0.2*dataset) + test=> 대충정답: 컬러사진


# 학습 및 검증 DataLoader 설정
train_dataloader = DataLoader(
    dataset=training_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=0  # 멀티프로세싱 비활성화
)

validation_dataloader = DataLoader(
    dataset=validation_dataset, 
    batch_size=CFG['BATCH_SIZE'], 
    shuffle=False, 
    num_workers=8, 
    pin_memory=True, 
    persistent_workers=True
)


In [36]:
# PatchGAN 기반의 Discriminator
class PatchGANDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(PatchGANDiscriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, kernel_size=4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        self.model = nn.Sequential(
            discriminator_block(in_channels * 2, 64, normalization=False),
            discriminator_block(64, 128),
            discriminator_block(128, 256),
            discriminator_block(256, 512),
            nn.Conv2d(512, 1, kernel_size=4, padding=1)
        )

    def forward(self, img_A, img_B):
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

# 가중치 초기화 함수
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [37]:
from segmentation_models_pytorch import UnetPlusPlus
import tqdm as notebook_tqdm

# EfficientNet-B4를 백본으로 사용
UNetPP = UnetPlusPlus(
    encoder_name="efficientnet-b4",  # pretrained on ImageNet
    encoder_weights="imagenet",     
    in_channels=3,
    classes=3
).to(device)

# generator로 UNetPP를 사용하도록 설정
generator = UNetPP

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from IPython.display import FileLink
import numpy as np
from skimage.metrics import structural_similarity as ssim

# 모델 저장을 위한 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)
log_file = os.path.join(model_save_dir, "training_log.txt")

# Best score와 loss 초기화
best_loss = float("inf")
best_score = float("-inf")
lambda_pixel = 100  # 픽셀 손실에 대한 가중치

# PatchGAN Discriminator 초기화
discriminator = PatchGANDiscriminator().to(device)

# 손실 함수 및 옵티마이저 설정
criterion_GAN = nn.MSELoss()
criterion_pixelwise = nn.L1Loss()

# Generator와 Discriminator 옵티마이저 설정
optimizer_G = optim.Adam(generator.parameters(), lr=CFG["LEARNING_RATE"])
optimizer_D = optim.Adam(discriminator.parameters(), lr=CFG["LEARNING_RATE"])

# 색상 히스토그램 유사도 함수
def histogram_similarity(pred, target, bins=256):
    pred_hist = cv2.calcHist([pred.cpu().numpy()], [0], None, [bins], [0, 256])
    target_hist = cv2.calcHist([target.cpu().numpy()], [0], None, [bins], [0, 256])
    hist_similarity = cv2.compareHist(pred_hist, target_hist, cv2.HISTCMP_CORREL)
    return hist_similarity

# 손실 함수
def combined_loss(fake_B, real_B):
    loss_GAN = criterion_GAN(fake_B, real_B)
    loss_pixel = criterion_pixelwise(fake_B, real_B)
    loss_ssim = 1 - ssim(fake_B, real_B, data_range=1.0)
    loss_hist = histogram_similarity(fake_B, real_B)
    return loss_GAN + lambda_pixel * loss_pixel + 0.4 * loss_ssim + 0.4 * loss_hist

def ssim_score(true, pred):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()  # NCHW → NHWC
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
    ssim_values = []
    for t, p in zip(true_np, pred_np):
        ssim_values.append(ssim(t, p, channel_axis=-1, data_range=p.max() - p.min()))
    return np.mean(ssim_values)

def masked_ssim_score(true, pred, mask):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()  # NCHW → NHWC
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
    mask_np = mask.cpu().numpy()  # 마스크를 NumPy 배열로 변환
    masked_ssim_values = []
    for t, p, m in zip(true_np, pred_np, mask_np):
        true_masked_pixels = t[m > 0]
        pred_masked_pixels = p[m > 0]
        masked_ssim_values.append(
            ssim(
                true_masked_pixels, 
                pred_masked_pixels, 
                channel_axis=-1, 
                data_range=pred_masked_pixels.max() - pred_masked_pixels.min()
            )
        )
    return np.mean(masked_ssim_values)

def histogram_similarity(true, pred):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)  # NCHW → NHWC
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)
    similarities = []
    for t, p in zip(true_np, pred_np):
        true_hsv = cv2.cvtColor(t, cv2.COLOR_RGB2HSV)
        pred_hsv = cv2.cvtColor(p, cv2.COLOR_RGB2HSV)
        hist_true = cv2.calcHist([true_hsv], [0], None, [180], [0, 180])
        hist_pred = cv2.calcHist([pred_hsv], [0], None, [180], [0, 180])
        hist_true = cv2.normalize(hist_true, hist_true).flatten()
        hist_pred = cv2.normalize(hist_pred, hist_pred).flatten()
        similarities.append(cv2.compareHist(hist_true, hist_pred, cv2.HISTCMP_CORREL))
    return np.mean(similarities)

# 학습 및 검증 루프
for epoch in range(1, CFG['EPOCHS'] + 1):
    generator.train()  # 학습 모드
    for i, batch in enumerate(train_dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        # Generator 훈련
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake).to(device))
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()

        # Discriminator 훈련
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real).to(device))
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake).to(device))
        loss_D = 0.5 * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()

        # Discriminator 정확도 계산
        real_accuracy = torch.mean((pred_real > 0.5).float())
        fake_accuracy = torch.mean((pred_fake < 0.5).float())
        accuracy = 0.5 * (real_accuracy + fake_accuracy)

        if i % 100 == 0:
            print(f"[Epoch {epoch}/{CFG['EPOCHS']}] [Batch {i}/{len(train_dataloader)}] "
                  f"[D loss: {loss_D.item()}] [G loss: {loss_G.item()}] [D accuracy: {accuracy.item() * 100:.2f}%]")

        # Validation 및 모델 저장
        if (i + 1) % 10 == 0 or i == len(train_dataloader) - 1:  # 10배치마다 또는 마지막 배치마다 저장
            try:
                generator.eval()
                validation_loss = 0.0
                with torch.no_grad():
                    for val_batch in validation_dataloader:
                        real_val_A = val_batch['A'].to(device)
                        real_val_B = val_batch['B'].to(device)
                        fake_val_B = generator(real_val_A)
                        val_loss = combined_loss(fake_val_B, real_val_B)
                        validation_loss += val_loss.item()

                validation_loss /= len(validation_dataloader)
                print(f"Validation loss at Epoch {epoch}, Batch {i + 1}: {validation_loss}")

                # 모델 저장
                if validation_loss < best_loss:
                    best_loss = validation_loss
                    torch.save(generator.state_dict(), os.path.join(model_save_dir, f"best_generator_epoch{epoch}_batch{i + 1}.pth"))
                    torch.save(discriminator.state_dict(), os.path.join(model_save_dir, f"best_discriminator_epoch{epoch}_batch{i + 1}.pth"))
                    print(f"Best model saved at Epoch {epoch}, Batch {i + 1} with Validation loss: {validation_loss}")
            except Exception as e:
                print(f"Error during validation or saving: {e}")


    # Validation 단계
    generator.eval()  # 검증 모드
    validation_loss = 0.0
    ssim_scores, masked_ssim_scores, hist_similarities = [], [], []
    with torch.no_grad():
        for val_batch in validation_dataloader:
            real_val_A = val_batch['A'].to(device)  # 손실 영역 마스크
            real_val_B = val_batch['B'].to(device)  # Ground Truth
            fake_val_B = generator(real_val_A)      # 복원된 이미지
    
            # 각각의 스코어 계산
            ssim_value = ssim_score(real_val_B, fake_val_B)
            masked_ssim_value = masked_ssim_score(real_val_B, fake_val_B, real_val_A)
            hist_value = histogram_similarity(real_val_B, fake_val_B)
    
            ssim_scores.append(ssim_value)
            masked_ssim_scores.append(masked_ssim_value)
            hist_similarities.append(hist_value)
    
            val_loss = combined_loss(fake_val_B, real_val_B)
            validation_loss += val_loss.item()
    
    # 평균 스코어 계산
    S = sum(ssim_scores) / len(ssim_scores)
    M = sum(masked_ssim_scores) / len(masked_ssim_scores)
    C = sum(hist_similarities) / len(hist_similarities)
    score = (0.2 * S) + (0.4 * M) + (0.4 * C)
    
    validation_loss /= len(validation_dataloader)
    validation_log_message = f"Validation loss: {validation_loss}, Score: {score}"
    print(validation_log_message)
    
    # 텍스트 파일에 Validation 손실 기록
    with open(log_file, "a") as log:
        log.write(validation_log_message + "\n")
    
    # SSIM 평가 기준에 따른 최적 모델 저장
    if score > best_score:
        best_score = score
        torch.save(generator.state_dict(), os.path.join(model_save_dir, "best_generator.pth"))
        torch.save(discriminator.state_dict(), os.path.join(model_save_dir, "best_discriminator.pth"))
        save_message = f"Best model saved at epoch {epoch} with Score: {score}"
        print(save_message)
        with open(log_file, "a") as log:
            log.write(save_message + "\n")

# 학습 완료 후 최고의 스코어 출력
final_message = f"Training completed. Best Score: {best_score}"
print(final_message)

[Epoch 1/2] [Batch 0/1481] [D loss: 0.4553558826446533] [G loss: 76.28998565673828] [D accuracy: 49.06%]


In [ ]:
import cv2
from PIL import Image
import numpy as np
import os

# 저장할 디렉토리 설정
model_save_dir = "./saved_models"
submission_dir = "./submission"
os.makedirs(submission_dir, exist_ok=True)

# 이미지 로드 및 전처리
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원을 추가합니다.
    return image

# 모델 경로 설정
generator_path = os.path.join("/kaggle/input/best-model01/best_generator (1).pth")

# 모델 로드 및 설정
# model = UNetPP(in_channels=3, out_channels=3).to(device)  # UNetPP로 설정
UNetPP.load_state_dict(torch.load(generator_path))
UNetPP.eval()

# 파일 리스트 불러오기
test_images = sorted(os.listdir(test_dir))

# 모든 테스트 이미지에 대해 추론 수행
for image_name in test_images:
    test_image_path = os.path.join(test_dir, image_name)

    # 손상된 테스트 이미지 로드 및 전처리
    test_image = load_image(test_image_path).to(device)

    with torch.no_grad():
        # 모델로 예측
        pred_image = UNetPP(test_image)
        pred_image = pred_image.cpu().squeeze(0)  # 배치 차원 제거
        pred_image = pred_image * 0.5 + 0.5  # 역정규화
        pred_image = pred_image.numpy().transpose(1, 2, 0)  # HWC로 변경
        pred_image = (pred_image * 255).astype('uint8')  # 0-255 범위로 변환
        
        # 예측된 이미지를 실제 이미지와 같은 512x512로 리사이즈
        pred_image_resized = cv2.resize(pred_image, (512, 512), interpolation=cv2.INTER_LINEAR)

    # 결과 이미지 저장
    output_path = os.path.join(submission_dir, image_name)
    cv2.imwrite(output_path, cv2.cvtColor(pred_image_resized, cv2.COLOR_RGB2BGR))    
    
print(f"Saved all images")


In [ ]:
# 저장된 결과 이미지를 ZIP 파일로 압축
zip_filename = "submission1.zip"
with zipfile.ZipFile(zip_filename, 'w') as submission_zip:
    for image_name in test_images:
        image_path = os.path.join(submission_dir, image_name)
        submission_zip.write(image_path, arcname=image_name)

print(f"All images saved in {zip_filename}")

In [ ]:
!tree